El siguiente libro requiere de los archivos presentes en el archivo "functions.py"

**Este libro es útil si lo que se busca es generar un .csv con el historial de partidas de 1 SOLO JUGADOR. Si se quiere analizar una población más grande, pasar al libro "population_scripts.ipynb". De ser el caso, lo único de interés aquí es la definición de la función "get_df_data" en el bloque 10**

La riot api key se reestablece cada 24 horas desde la página de la api de riot games

Importante recordar el límite de request a la api de 5 cada 2 segundos y 100 cada 2 miutos

In [1]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import time
load_dotenv()

True

Se introduce el nombre y el tagLine de la cuenta de RIOT a fin de obtener el PUUID asociado.

In [2]:
gameName = 'Natanaël%20Cano'  #El %20 indica un espacio
tagLine = 'NATA'
api_key = os.getenv('RIOT_API_KEY') #Se actializa cada 24 horas

In [3]:
from functions import get_puuid

puuid = get_puuid(gameName=gameName, tagLine=tagLine, api_key=api_key)
"""
Se obtiene el PUUID del jugador seleccionado
"""
print(puuid)

jsPdXjm4uDyQz5yxApVwAEr-NeF8x8QRgzto0VcFyFe0RoJID_ZYQBFXujbry4kwUPXm5UhZXVmn6Q


El siguiente bloque se utiliza para buscar el historial de partidas de un jugador. Por defecto utilizamos matchtype = 'ranked' para analizar sólo partidas competitivas, pero se aceptan los siguienets argumentos:

matchtype = { ranked, normal, tourney, tutorial}

Sin embargo, esto muestra TODAS las partidas ranked. Para filtrar sólo partidas de SoloQ, se añade el queue id. Lista completa en:
https://static.developer.riotgames.com/docs/lol/queues.json

queueid = {420 = soloq, 440 = flex 5v5} son las colas clasificatorias actuales.

In [ ]:
matchtype = 'ranked'
queueid = '420' #Default para SoloQ
start = '0'
count = '5'

In [5]:
from functions import get_matchids
history = get_matchids(puuid=puuid, queueid=queueid, matchtype=matchtype, start=start, count=count, api_key=api_key)
print(history)

['LA1_1642972444', 'LA1_1642962195', 'LA1_1642710668', 'LA1_1642704329', 'LA1_1642695796']


El siguiente bloque es "el más importante" dado que reune toda la información recopilada hasta el momento. En orden de aparición, enlista:

* match id : 			        id de la partida
* my team : 			        equipo al que pertenece el jugador
* gold min 10 (opcional) :	Oro del jugador al minuto 10
* team1 gold : 			    Oro del equipo 100 al min 10
* team2 gold : 			    Oro del equipo 200 al min 10
* gold diff :			        Diferencia de oro a minuto 10
* gold leading team : 		A favor de qué equipo es la diferencia de oro
* my team ahead gold : 		El equipo del jugador es el de la ventaja. True o False.
* team 100 kills :		    kills del equipo 100 al min 10
* team 200 kills : 		    kills del equipo 200 al min 10
* kill diff :			        Diferencia de kills a minuto 10
* kill leading team : 		A favor de qué equipo es la diferencia de kills
* my team ahead kills : 		El equipo del jugador es el de la ventaja. True o False.
* my team win : 			    El equipo del jugador gana. True o False

In [6]:
from functions import get_participant_info, get_gold_at_min10, get_team_gold_difference, get_team_kills_at_10, get_winner

csv_path = "match_stats_min10_NATA.csv"
"""
Con esta info se genera un dataframe y se envían los NO duplicados a un .csv
"""
if os.path.exists(csv_path):
        df_old = pd.read_csv(csv_path)
        data = df_old.to_dict(orient="records")
        existing_match_ids = {row["match_id"] for row in data}
else:
        data = []
        existing_match_ids = set()

    # --- Procesar cada partida ---
for match_id in history:
        if match_id in existing_match_ids:
            print(f"[{match_id}] Ya estaba registrado, se omite.")
            continue

        try:
            # URLs
            match_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
            timeline_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"

            # Requests
            match_data = requests.get(match_url).json()
            timeline_data = requests.get(timeline_url).json()

            # Info del jugador
            participant_id, team_id = get_participant_info(match_data, puuid, mode = 'both')
            if not participant_id:
                print(f"[{match_id}] PUUID no encontrado.")
                continue

            my_team = get_participant_info(match_data, puuid, mode = 'team')

            # Determinar victoria
            player_info = next((p for p in match_data["info"]["participants"] if p["puuid"] == puuid), None)
            if not player_info:
                print(f"[{match_id}] No se encontró info del jugador.")
                continue
            my_team_win = player_info["win"]

            # Oro por equipo
            gold_info = get_team_gold_difference(timeline_data, my_team)
            if not gold_info:
                print(f"[{match_id}] Error al obtener oro de equipos.")
                continue

            # Kills por equipo
            kill_info = get_team_kills_at_10(timeline_data, my_team)
            if not kill_info:
                print(f"[{match_id}] Error al obtener kills")
                continue

            # Mi equipo
            my_team_label = 100 if team_id == 100 else 200
            my_team_was_ahead_kills = (team_id == 100 and kill_info['kill_diff'] > 0) or \
                                      (team_id == 200 and kill_info['kill_diff'] < 0)
            my_team_was_ahead_gold = (team_id == 100 and gold_info["leading_team"] == 1) or \
                                     (team_id == 200 and gold_info["leading_team"] == 2)

            # Ganador
            winner = get_winner(match_data)

            # Agregar fila
            data.append({
                "match_id": match_id,
                "my_team": my_team_label,
                "team1_gold": gold_info["team1_gold"],
                "team2_gold": gold_info["team2_gold"],
                "gold_diff": gold_info["difference"],
                "gold_leading_team": gold_info["leading_team_id"],
                "my_team_ahead_gold": my_team_was_ahead_gold,
                "team1_kills": kill_info['team1_kills'],
                "team2_kills": kill_info['team2_kills'],
                "kill_diff": kill_info['kill_diff'],
                "kill_leading_team": kill_info['kill_leading_team'],
                "my_team_ahead_kills": my_team_was_ahead_kills,
                "my_team_win": my_team_win,
                "winner": winner
            })

            time.sleep(1)  # Evitar rate limit
        except Exception as e:
            print(f"[{match_id}] Error: {e}")

**NOTA:** El bloque anterior se encuentra también en el archivo "functions.py" con nombre get_df_data.

Ahora con esta información se genera un dataframe (df) y se guarda la info en el .csv "match_stats_min10"

In [7]:
df = pd.DataFrame(data)
df.to_csv(csv_path, index=False)
print(f"Guardado en {csv_path}")

Guardado en match_stats_min10_NATA.csv


In [8]:
print(len(data))

5


Se lee el csv y se genera un dataframe con toda la info acumulada, df.tail() muesta sólo los últimos elementos del df, a fin de comprobar que se estén almacenando correctamente

In [9]:
df = pd.read_csv("match_stats_min10_NATA.csv")
df.tail()

,match_id,my_team,team1_gold,team2_gold,gold_diff,gold_leading_team,my_team_ahead_gold,team1_kills,team2_kills,kill_diff,kill_leading_team,my_team_ahead_kills,my_team_win,winner
0,LA1_1642972444,200,17258,16582,-676,100,False,7,7,0,0,False,False,100
1,LA1_1642962195,200,18372,15515,-2857,100,False,8,3,-5,100,True,False,100
2,LA1_1642710668,200,16587,14087,-2500,100,False,6,2,-4,100,True,False,100
3,LA1_1642704329,200,16754,16985,231,200,False,7,8,1,200,False,True,200
4,LA1_1642695796,200,16144,17291,1147,200,False,7,8,1,200,False,False,100


In [10]:
df

,match_id,my_team,team1_gold,team2_gold,gold_diff,gold_leading_team,my_team_ahead_gold,team1_kills,team2_kills,kill_diff,kill_leading_team,my_team_ahead_kills,my_team_win,winner
0,LA1_1642972444,200,17258,16582,-676,100,False,7,7,0,0,False,False,100
1,LA1_1642962195,200,18372,15515,-2857,100,False,8,3,-5,100,True,False,100
2,LA1_1642710668,200,16587,14087,-2500,100,False,6,2,-4,100,True,False,100
3,LA1_1642704329,200,16754,16985,231,200,False,7,8,1,200,False,True,200
4,LA1_1642695796,200,16144,17291,1147,200,False,7,8,1,200,False,False,100
